In [1]:
from sys import exit
from Crypto.Util.number import bytes_to_long, inverse, long_to_bytes
import random
import string
from pwn import *
CHARSET = string.ascii_uppercase + string.ascii_lowercase + string.digits

import numpy as np
import math

## **Bài toán GUESS MY CHEESE (Part 1)**

Bài toán này yêu cầu chúng ta giải mã đoạn mã đã cho với các thuật toán liên quan đến tuyến tính bao gồm:
- **Caesar Cipher**
- **Affine Cipher**
- **Vigenère Cipher**
- **Hill Cipher**

In [2]:
def affine_encrypt(text, a, b):
    """Mã hóa Affine Cipher với công thức: E(x) = (a*x + b) mod 26"""
    encrypted_text = ""
    for char in text.upper():
        if char.isalpha():
            x = ord(char) - ord('A')
            encrypted_text += chr(((a * x + b) % 26) + ord('A'))
        else:
            encrypted_text += char
    return encrypted_text

def affine_decrypt(text, a, b):
    """Giải mã Affine Cipher bằng cách tìm nghịch đảo của a."""
    decrypted_text = ""
    mod_inv_a = pow(a, -1, 26)  # Tìm nghịch đảo modulo của a
    for char in text.upper():
        if char.isalpha():
            y = ord(char) - ord('A')
            decrypted_text += chr(((mod_inv_a * (y - b)) % 26) + ord('A'))
        else:
            decrypted_text += char
    return decrypted_text

def caesar_variant_encrypt(text, shift_pattern):
    """Mã hóa Caesar với độ dịch chuyển thay đổi theo một pattern."""
    encrypted_text = ""
    for i, char in enumerate(text.upper()):
        if char.isalpha():
            shift = shift_pattern[i % len(shift_pattern)]  # Lặp lại pattern
            encrypted_text += chr(((ord(char) - ord('A') + shift) % 26) + ord('A'))
        else:
            encrypted_text += char
    return encrypted_text

def caesar_variant_decrypt(text, shift_pattern):
    """Giải mã Caesar bằng cách dịch ngược lại theo pattern."""
    decrypted_text = ""
    for i, char in enumerate(text.upper()):
        if char.isalpha():
            shift = shift_pattern[i % len(shift_pattern)]
            decrypted_text += chr(((ord(char) - ord('A') - shift) % 26) + ord('A'))
        else:
            decrypted_text += char
    return decrypted_text

def columnar_transposition_encrypt(text, key):
    """Mã hóa Columnar Transposition dựa trên một key nhất định."""
    text = text.replace(" ", "")  # Loại bỏ khoảng trắng
    num_cols = len(key)
    num_rows = math.ceil(len(text) / num_cols)
    matrix = [['' for _ in range(num_cols)] for _ in range(num_rows)]
    
    # Điền ma trận theo hàng ngang
    index = 0
    for r in range(num_rows):
        for c in range(num_cols):
            if index < len(text):
                matrix[r][c] = text[index]
                index += 1
    
    # Đọc theo thứ tự cột dựa trên key
    sorted_key = sorted([(val, i) for i, val in enumerate(key)])
    encrypted_text = ""
    for _, col in sorted_key:
        for row in matrix:
            if row[col]:
                encrypted_text += row[col]
    return encrypted_text

def columnar_transposition_decrypt(text, key):
    """Giải mã Columnar Transposition dựa trên key."""
    num_cols = len(key)
    num_rows = math.ceil(len(text) / num_cols)
    matrix = [['' for _ in range(num_cols)] for _ in range(num_rows)]
    
    sorted_key = sorted([(val, i) for i, val in enumerate(key)])
    index = 0
    for _, col in sorted_key:
        for r in range(num_rows):
            if index < len(text):
                matrix[r][col] = text[index]
                index += 1
    
    decrypted_text = "".join("".join(row) for row in matrix).strip()
    return decrypted_text

In [3]:
# Có các cặp dữ liệu sau:
plain_lst = ["MOZZARELLA", "RICOTTA"]
cipher_lst = ["ZFMMPOBWWP", "FAOMJJK"]

def find_affine_key(plain_text, cipher_text):
    """Tìm giá trị a, b trong Affine Cipher dựa trên hai cặp ký tự đầu tiên."""
    def char_to_num(char):
        return ord(char) - ord('A')
    
    p1, p2 = char_to_num(plain_text[0]), char_to_num(plain_text[1])
    c1, c2 = char_to_num(cipher_text[0]), char_to_num(cipher_text[1])
    
    solutions = []
    for a in range(1, 26, 2):  # a phải có nghịch đảo modulo 26 => số lẻ trừ 13
        if math.gcd(a, 26) == 1:
            b = (c1 - a * p1) % 26
            if (a * p2 + b) % 26 == c2:
                solutions.append((a, b))
    
    return solutions

find_affine_key(plain_lst[0], cipher_lst[0])

# affine_encrypt(plain_lst[1], 15, 10)
affine_decrypt("MPZFOPCFNFD", 3, 15)

'ZAMORANOIOW'

In [ ]:
# Code hoàn chỉnh để giải mã vì đây là mã Affine Cipher:
plain_lst = ["MOZZARELLA", "RICOTTA"]
cipher_lst = []

port = 59244
r = remote('verbal-sleep.picoctf.net', port)

get_success = r.recvuntil("to guess it:  ").decode()
cipher_text = r.recvline(keepends=False).strip().decode()
# r.sendline('1337')

get_success = r.recvuntil("What would you like to do?").decode()
get_success = r.recvline(keepends=False).strip().decode()

r.sendline('e')
get_success = r.recvuntil("would you like to encrypt? ").decode()
r.sendline(plain_lst[0])
get_success = r.recvuntil("Here's your encrypted cheese:  ").decode()
cipher_lst.append(r.recvline(keepends=False).strip().decode())

ab_lst = find_affine_key(plain_lst[0], cipher_lst[0])
a, b = ab_lst[0]
print(a,b)
decrypted_text = affine_decrypt(cipher_text, a, b)
get_success = r.recvuntil("What would you like to do?").decode()

r.sendline('g')
get_success = r.recvuntil("So...what's my cheese?").decode()
get_success = r.recvline(keepends=False).strip().decode()
r.sendline(decrypted_text)

get_success = r.recvuntil("Here's the password to the cloning room:  ").decode()
flag = r.recvline(keepends=False).strip().decode()
print(flag)     #picoCTF{ChEeSy1ff8ae0d}

# print(get_success)

[x] Opening connection to verbal-sleep.picoctf.net on port 59244
[x] Opening connection to verbal-sleep.picoctf.net on port 59244: Trying 3.138.217.147
[+] Opening connection to verbal-sleep.picoctf.net on port 59244: Done


C:\Users\Dell\AppData\Local\Temp\ipykernel_11848\1796535688.py:8: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  get_success = r.recvuntil("to guess it:  ").decode()
C:\Users\Dell\AppData\Local\Temp\ipykernel_11848\1796535688.py:12: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  get_success = r.recvuntil("What would you like to do?").decode()
C:\Users\Dell\AppData\Local\Temp\ipykernel_11848\1796535688.py:15: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendline('e')
C:\Users\Dell\AppData\Local\Temp\ipykernel_11848\1796535688.py:16: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  get_success = r.recvuntil("would you like to encrypt? ").decode()
C:\Users\Dell\AppData\Local\Temp\ipykernel_11848\1796535688.py:17: BytesWarning: Text is not bytes; assuming ASCII, no gu

11 5


C:\Users\Dell\AppData\Local\Temp\ipykernel_11848\1796535688.py:25: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  get_success = r.recvuntil("What would you like to do?").decode()
C:\Users\Dell\AppData\Local\Temp\ipykernel_11848\1796535688.py:27: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendline('g')
C:\Users\Dell\AppData\Local\Temp\ipykernel_11848\1796535688.py:28: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  get_success = r.recvuntil("So...what's my cheese?").decode()
C:\Users\Dell\AppData\Local\Temp\ipykernel_11848\1796535688.py:30: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendline(decrypted_text)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11848\1796535688.py:32: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwn

picoCTF{ChEeSy1ff8ae0d}
